In [1]:
import iResNet

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

import importlib
importlib.reload(iResNet)

<module 'iResNet' from '/home/yanbo/Research/Lab/eLIFE/iResNetLab/src/iResNet.py'>

## FCN

In [22]:
model = iResNet.FCN(2, 2)
model

FCN(
  (net): SNFCN(
    (g): Sequential(
      (0): Linear(in_features=2, out_features=16, bias=True)
      (1): GELU()
      (2): Linear(in_features=16, out_features=16, bias=True)
      (3): GELU()
      (4): Linear(in_features=16, out_features=2, bias=True)
    )
  )
  (noise): NormalDistribution()
)

In [23]:
x = torch.Tensor([[1,2], [3,4]])
x.requires_grad = True
model.train()

y, p, logdet = model(x)
y

tensor([[1.3179, 2.2021],
        [3.3864, 4.2015]], grad_fn=<SliceBackward>)

In [26]:
model.inverse(y.detach())

tensor([[1., 2.],
        [3., 4.]])

## Sequential

In [27]:
model = iResNet.Sequential(iResNet.FCN(2, 2),
                           iResNet.FCN(2, 2),
                           iResNet.FCN(2, 2))
print(model)

Sequential(
  (0): FCN(
    (net): SNFCN(
      (g): Sequential(
        (0): Linear(in_features=2, out_features=16, bias=True)
        (1): GELU()
        (2): Linear(in_features=16, out_features=16, bias=True)
        (3): GELU()
        (4): Linear(in_features=16, out_features=2, bias=True)
      )
    )
    (noise): NormalDistribution()
  )
  (1): FCN(
    (net): SNFCN(
      (g): Sequential(
        (0): Linear(in_features=2, out_features=16, bias=True)
        (1): GELU()
        (2): Linear(in_features=16, out_features=16, bias=True)
        (3): GELU()
        (4): Linear(in_features=16, out_features=2, bias=True)
      )
    )
    (noise): NormalDistribution()
  )
  (2): FCN(
    (net): SNFCN(
      (g): Sequential(
        (0): Linear(in_features=2, out_features=16, bias=True)
        (1): GELU()
        (2): Linear(in_features=16, out_features=16, bias=True)
        (3): GELU()
        (4): Linear(in_features=16, out_features=2, bias=True)
      )
    )
    (noise): NormalDi

In [18]:
x = torch.Tensor([[1,2], [3,4], [-3,1]])
x.requires_grad = True
model.train()

y, p, logdet = model(x)
y

tensor([[ 0.9652,  2.7492],
        [ 3.3913,  4.7129],
        [-2.4317,  1.4621]], grad_fn=<SliceBackward>)

In [19]:
x = torch.Tensor([[1,2], [3,4], [-2, 1]])
x.requires_grad = True
model.eval()

y = model(x)
y

tensor([[ 0.9652,  2.7492],
        [ 3.3913,  4.7129],
        [-1.6966,  1.5556]], grad_fn=<SliceBackward>)

In [20]:
model.inverse(y.detach())

tensor([[ 1.0000,  2.0000],
        [ 3.0000,  4.0000],
        [-2.0000,  1.0000]])

# Conv1d

In [2]:
#model = Conv1d(2, 3)
model = iResNet.Sequential(iResNet.Conv1d(channel=2, kernel_size=3),
                           iResNet.Conv1d(channel=2, kernel_size=1),
                           iResNet.Conv1d(channel=2, kernel_size=3))
model

Sequential(
  (0): Conv1d(
    (net): SNCov1d(
      (net): Sequential(
        (0): Conv1d(2, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): GELU()
        (2): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (3): GELU()
        (4): Conv1d(16, 2, kernel_size=(3,), stride=(1,), padding=(1,))
      )
    )
  )
  (1): Conv1d(
    (net): SNCov1d(
      (net): Sequential(
        (0): Conv1d(2, 16, kernel_size=(1,), stride=(1,))
        (1): GELU()
        (2): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
        (3): GELU()
        (4): Conv1d(16, 2, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (2): Conv1d(
    (net): SNCov1d(
      (net): Sequential(
        (0): Conv1d(2, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): GELU()
        (2): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (3): GELU()
        (4): Conv1d(16, 2, kernel_size=(3,), stride=(1,), padding=(1,))
      )
    )
  )
)

In [7]:
x = torch.Tensor([[[1,2,3,4,5],[-1,-2,-3,-4,-5]], [[1,2,0,4,5],[-1,-2,0,-4,-5]]])
x.requires_grad = True

y, logp, logdet = model(x)
y

tensor([[[ 0.9487,  1.8784,  2.7273,  3.6926,  4.6764],
         [-1.1954, -2.2772, -3.3897, -4.4579, -5.3149]],

        [[ 0.9672,  1.9152, -0.1993,  3.7975,  4.6542],
         [-1.1567, -2.2055, -0.2905, -4.4541, -5.3364]]],
       grad_fn=<AddBackward0>)

In [8]:
model.inverse(y)

tensor([[[ 1.0000e+00,  2.0000e+00,  3.0000e+00,  4.0000e+00,  5.0000e+00],
         [-1.0000e+00, -2.0000e+00, -3.0000e+00, -4.0000e+00, -5.0000e+00]],

        [[ 1.0000e+00,  2.0000e+00, -1.4901e-08,  4.0000e+00,  5.0000e+00],
         [-1.0000e+00, -2.0000e+00, -7.4506e-09, -4.0000e+00, -5.0000e+00]]])